# Data Cleaning und Handling


### Imports

In [40]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np

## Code

### Dowloading Data

In [41]:
#response = requests.get("https://github.com/MarcusGrum/AIBAS/blob/main")
#response = requests.get("https://www.kaggle.com/datasets/victormegir/bikes-from-bikezcom/data")

#doc = BeautifulSoup(response.text, "html.parser")

#article = doc.find("article")

#print(doc)
#print("testpush")
#table = article.find("table")

# df = pd.read_html(str(table))[0].copy()
# print(df)


"""
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "bikes.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "victormegir/bikes-from-bikezcom",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("victormegir/bikes-from-bikezcom")

print("Path to dataset files:", path)
"""

'\nimport kagglehub\nfrom kagglehub import KaggleDatasetAdapter\n\n# Set the path to the file you\'d like to load\nfile_path = "bikes.csv"\n\n# Load the latest version\ndf = kagglehub.dataset_load(\n  KaggleDatasetAdapter.PANDAS,\n  "victormegir/bikes-from-bikezcom",\n  file_path,\n  # Provide any additional arguments like \n  # sql_query or pandas_kwargs. See the \n  # documenation for more information:\n  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas\n)\n\nimport kagglehub\n\n# Download latest version\npath = kagglehub.dataset_download("victormegir/bikes-from-bikezcom")\n\nprint("Path to dataset files:", path)\n'

### Cleaning Up the Data

### Drop NA

In [42]:
current=current = os.getcwd()
parent=parent_dir = os.path.dirname(current)

df=pd.read_csv(parent_dir+"/datasets/victormegir/bikes-from-bikezcom/versions/4/bikes.csv")

##---------------Sichere Auswahl mit DropNA---------------------##

kat_selection_safe=["Category","Engine type","Transmission type,final drive","Displacement ccm","Fuel capacity liters","Power HP","Front brakes","Rear brakes"]
kat_selection_all=["Category","Engine type","Transmission type,final drive","Displacement ccm","Fuel capacity liters","Power HP","Front brakes","Rear brakes","Starter","Seat height mm","Dry weight kg","Overall length mm","Wheelbase mm","Top speed km/h"]

df_selected_safe=df[kat_selection_safe].copy()
df_selected_all=df[kat_selection_all].copy()

#print(df_all.shape)

df_clean=df_selected_safe.dropna().copy()
df_all_0=df_selected_all.fillna(0).copy()

print(df_all_0.head())

       Category                   Engine type Transmission type,final drive  \
0         Sport  Single cylinder, four-stroke                         Chain   
1  Super motard               V2, four-stroke                         Chain   
2         Sport  Single cylinder, four-stroke                         Chain   
3         Sport  Single cylinder, four-stroke                         Chain   
4         Sport   Single cylinder, two-stroke                         Chain   

   Displacement ccm  Fuel capacity liters  Power HP  \
0             600.0                  17.0       0.0   
1             896.1                  12.0      95.2   
2             124.2                  14.5      28.2   
3             124.2                  14.5      15.0   
4              50.0                  14.5       0.0   

                                        Front brakes  \
0              Single disc. Brembo 2-piston calipers   
1  Double disc. ABS.Stainless steel wave floating...   
2  Single disc. ABS. Stain

C:\Users\alexa\AppData\Local\Temp\ipykernel_16164\751862266.py:4: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(parent_dir+"/datasets/victormegir/bikes-from-bikezcom/versions/4/bikes.csv")


### Editing values

In [43]:
##-------Bis zum Punkt steht doppel/Single etc-------------##
df_clean["Front brakes"]=df_clean['Front brakes'].str.split('.').str[0]
df_clean["Rear brakes"]=df_clean['Rear brakes'].str.split('.').str[0]
print(df_clean.shape)
# mindest Wert denn eine Art haben muss an Datensatzhäufigkeit
min_count = 200

# Häufigkeiten zählen
value_counts = df_clean['Category'].value_counts()
#print(value_counts)

# Kategorien mit genug Vorkommen identifizieren
valid_categories = value_counts[value_counts >= min_count].index
#print(valid_categories)

# DataFrame filtern
df_clean = df_clean[df_clean['Category'].isin(valid_categories)].copy()
print(df_clean.shape)
print(df_clean)


(18847, 8)
(18573, 8)
           Category                      Engine type  \
1      Super motard                  V2, four-stroke   
2             Sport     Single cylinder, four-stroke   
3             Sport     Single cylinder, four-stroke   
5             Sport                Twin, four-stroke   
6             Sport                Twin, four-stroke   
...             ...                              ...   
38240    Naked bike  Two cylinder boxer, four-stroke   
38272    Naked bike  Two cylinder boxer, four-stroke   
38305    Naked bike  Two cylinder boxer, four-stroke   
38339    Naked bike  Two cylinder boxer, four-stroke   
38365         Sport      Single cylinder, two-stroke   

      Transmission type,final drive  Displacement ccm  Fuel capacity liters  \
1                             Chain             896.1                  12.0   
2                             Chain             124.2                  14.5   
3                             Chain             124.2               

In [44]:
numeric_columns = df_clean.select_dtypes(include=['number']).columns.tolist()

def removeOutliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1

    print("IQR value for column %s is: %s" % (col, IQR))
    global outlier_free_list
    global filtered_data

    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    outlier_free_list = [x for x in data[col] if (
        (x > lower_range) & (x < upper_range))]
    filtered_data = data.loc[data[col].isin(outlier_free_list)] #entsprechende Outlier free rows setzen

counter=0
for i in numeric_columns:
    if counter == 0:
      removeOutliers(df_clean, i)
      counter=counter+1 # einaml mit anfangs df rein, danach mit arbeist variable
    else: 
      removeOutliers(filtered_data, i)

df_clean=filtered_data
print(df_clean.shape)

IQR value for column Displacement ccm is: 735.0
IQR value for column Fuel capacity liters is: 7.0
IQR value for column Power HP is: 68.0
(17972, 8)
